In [2]:
import pandas as pd
import os
from fastai.vision.all import *
from PIL import Image
import torchvision.transforms as transforms
from torchvision import transforms

C:\Users\bowan\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
Image_path = '/images_assorted'

if os.path.exists(Image_path):
    print("The file exists.")
else:
    %run GenerateVolumes.ipynb

2
3
4
5
6
7
8
9
10
11
12



KeyboardInterrupt



KeyboardInterrupt: 

In [2]:

df = pd.read_csv('volume_data_assorted.csv')
df.head()

,Radius,X_Position,Y_Position,Z_Position,Volume,image_path,shape
0,14,28,50,35,2744,images_assorted/image_1_slicenumber.png,sphere
1,18,44,42,25,5832,images_assorted/image_2_slicenumber.png,sphere
2,14,26,37,22,2744,images_assorted/image_3_slicenumber.png,cube
3,8,57,33,55,512,images_assorted/image_4_slicenumber.png,cube
4,11,33,48,67,1331,images_assorted/image_5_slicenumber.png,sphere


In [3]:
# Keep the first 50% of rows
#df = df.iloc[:len(df) // 5]



In [4]:

def get_target(row):
    # Convert image path to PILImage
    target_data = row['shape']
    return target_data

In [5]:
transform = transforms.ToTensor()
def get_inputs(row):
    #print("stacking now")
    channels = 3
    stacked_tensor = torch.zeros((channels, 100, 100, 100))  # Shape [3, 100, 100, 100]
    image_path = row['image_path']
    slicenumber = 0
    while slicenumber < 100:
        # Replace 'slicenumber' with the current slice number in the path
        image_path_slice = image_path.replace("slicenumber", str(slicenumber))
        image_data_slice = PILImage.create(image_path_slice)

        # Apply transformation to the slice
        image_tensor_slice = transform(image_data_slice)  # Shape [3, 100, 100]

        # Insert the transformed slice into the stacked tensor
        stacked_tensor[:, :, :, slicenumber] = image_tensor_slice  # Fill along the depth axis

        #print(f"Processed slice {slicenumber}: Stacked tensor shape: {stacked_tensor.shape}")
        slicenumber += 1 
    #print("stacking finshed")
    return stacked_tensor 

In [6]:
from fastai.data.transforms import Transform

class AnyShapeTransform(Transform):
    def encodes(self, x):
        # Example: Ensure tensor compatibility with the model
        return x.float()  # Convert to float32 (or any required type)
    
custom_block = TransformBlock(type_tfms=AnyShapeTransform())

In [7]:
dblock = DataBlock(
    blocks=(custom_block, CategoryBlock),  # Image input and auxiliary input
    get_x=get_inputs,  # Get both the image path and the auxiliary input
    get_y=get_target  # Get the radius value from the dataframe
    #splitter=RandomSplitter(valid_pct=0.2, seed=42),  # Split into train and validation sets
    #item_tfms=Resize(512)  # Resize images to 512x512

)


In [8]:
#os.chdir('..') # just moves directory where shared dataset is
dls = dblock.dataloaders(df, bs=4,num_workers=24)


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResNet3D(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet3D, self).__init__()
        
        # Initial convolution
        self.conv1 = nn.Conv3d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)

        # Residual blocks
        self.layer1 = self._make_layer(64, 64, 3)  # 3 residual blocks
        self.layer2 = self._make_layer(64, 128, 4, stride=2)  # Downsampling
        self.layer3 = self._make_layer(128, 256, 6, stride=2)  # Downsampling
        self.layer4 = self._make_layer(256, 512, 3, stride=2)  # Downsampling

        # Global Average Pooling
        self.global_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        
        # Fully connected layer for classification
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = []
        # First block with downsampling
        layers.append(ResidualBlock3D(in_channels, out_channels, stride))
        # Remaining blocks
        for _ in range(1, blocks):
            layers.append(ResidualBlock3D(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Initial layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Residual layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Global pooling and fully connected layer
        x = self.global_pool(x)
        x = torch.flatten(x, 1)  # Flatten for FC layer
        x = self.fc(x)
        return x

class ResidualBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock3D, self).__init__()
        
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        # Identity mapping for residual connection
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm3d(out_channels)
            )
        
    def forward(self, x):
        identity = x  # Save input for residual connection
        
        # Residual block
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Add identity connection
        if self.downsample is not None:
            identity = self.downsample(identity)
        out += identity
        out = self.relu(out)
        
        return out


In [11]:
from fastai.learner import Learner
# Create and train the learner
model = ResNet3D()  # Note the parentheses , metrics=rmse
learn = Learner(dls, model, metrics=accuracy)


In [12]:
#learn.fit_one_cycle(15,lr_max=lr_steep)

learn.fit_one_cycle(50)

epoch,train_loss,valid_loss,accuracy,time
0,0.409395,0.073109,0.980000,00:22
1,0.268894,0.076632,0.990000,00:22
2,0.300455,0.014031,1.000000,00:23
3,0.252274,1.041082,0.630000,00:23
4,0.316952,1.060918,0.700000,00:24
5,0.257356,0.103453,0.990000,00:23
6,0.608267,0.694575,0.520000,01:57
7,0.454473,2.081613,0.480000,00:27
8,0.304687,1.095857,0.650000,02:45
9,0.295450,0.093626,0.980000,01:11


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
print(transform)